# Daten visualisieren

In diesem Notebook lernst du anhand von Beispielen, wie du große Datenmengen mit Python visualisieren kannst.

Du lernst nützliche Python-Bibliotheken und den Umgang damit kennen.

Du lernst verschiedene Darstellungsformen und deren Vor- und Nachteile kennen.

## Importieren der Bibliotheken und Einlesen der Daten

Zunächst importieren wir alle **Bibliotheken**, die wir im Folgendem benötigen werden. Anschließend lesen wir die **bereinigten Daten** aus der letzten Phase ein.

In [ ]:
# Import der Bibliotheken
import pandas as pd
import numpy as np
import cufflinks as cf
import ipywidgets as widgets
from ipywidgets import interact
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

In [ ]:
# Einlesen der Daten
# Interpretiere die Spalte TimeStamp als Datum (parse_dates)
df = pd.read_csv(filepath_or_buffer='Daten/EinlauefeBereinigt.csv', parse_dates=['TimeStamp'])
# Setzen des Index
df = df.set_index(keys='TimeStamp')
# Ausgabe der ersten fuenf Zeilen
df.head()

## Park-Daten eines bestimmten Wochentags herausfiltern und visualisieren

Wir beginnen damit einen ausgewählten Tag (Montag der 06.03.2017) aus den Daten herauszufiltern.

In [ ]:
montag = df['2017-03-06']

Anschließend zeichnen wir mittels der Funktion `iplot()` den Graphen über die entsprechenden Daten.

Mit `xTitle` und `yTitle` können wir die Achsen beschriften. Mit `title` legen wir die Überschrift fest.

In [ ]:
montag.iplot(
    title='Montag der 06.03.2017',
    xTitle='Uhrzeit',
    yTitle='Belegung'
)

Cufflinks wählt auomatisch die Darstellung in einem sogenannten **Scatter-Plot** und verbindet die einzelnen Datenpunkte paarweise druch eine gerade Linie.

Da die Belegungsdaten nur stündlich vorliegen können wir jedoch keine genauen Aussagen über die Belegungen zwischen zwei Messpunkten machen.

Eine alternative Form der Darstellung, die diesen Umstand berücksichtigt ist die Darstellung in einem **Säulendiagramm**.

In [ ]:
montag.iplot(
    title='Montag der 06.03.2017',
    kind='bar',
    xTitle='Uhrzeit',
    yTitle='Belegung'
)

Diese Art der Darstellung ist jedoch sehr unübersichtlich. Daher möchten wir im Folgendem die Graphen in sogenannte **Subplots** aufteilen.

Wichtig dabei ist, dass diese sich die **selbe Achse teilen**.
Zusätzlich beschränken wir uns nun auf die Parklätze P1, P2 und P3, um diese miteinander zu vergleichen.

In [ ]:
# Beachte die unterschiedlichen Skalierungen der y-Achsen
montag.iplot(
    y=['P1AK', 'P2AK', 'P3AK'],
    title='Montag der 06.03.2017',
    kind='bar',
    yrange=[0,600],
    subplots=True,
    shared_xaxes=True
)

<div style="background-color: lightblue; padding: 5px 20px 20px">

### Aufgabe (Relative Belegung)

Anstatt der **absoluten Anzahl** an Fahrzeugen soll die Belegung der Parkplätze P1, P2 und P3 für eine bessere Vergleichbarkeit **prozentual** in Abhängigkeit von der **maximalen Kapazität** des jeweiligen Parkplatzes angezeigt werden. Überarbeite dazu den Code aus der vorherigen Zelle in der nachfolgenden Zelle.

Tipp: Erzeuge zunächst einen neuen `DataFrame` mit Namen `df_rel`, der die Belegung prozentual abspeichert. Die maximalen Kapazitäten der Parkplätze und ein Beispiel findest du in dem vorherigen Notebook.
</div>

In [ ]:
# Hier ist Platz für deinen Code

### Interaktive Ausgabe mit `ipywidgets`

Mit Hilfe sogenannter `ipywidgets` können wir interaktive Eingabemöglichkeiten schaffen.

Hier definieren wir einen sogenannten `DatePicker` mit dessen Hilfe wir ein Datum eingeben können.

In [ ]:
# Definiere den DatePicker
date_picker = widgets.DatePicker(
    value=pd.datetime(year=2017, month=1, day=1),
    description='Datum'
)
# Definiere Ausgabefeld
out = widgets.Output()

# Funktion, die aufgerufen wird, wenn wir das Datum ändern
def plot(value):
    date = value.new
    out.clear_output()
    with out:
        df[df.index.date==date].iplot(xTitle='Uhrzeit', yTitle='Belegung')

# Überwache Änderungen am DatePicker
date_picker.observe(plot, names='value')

# Zeige beide Widgets an
display(date_picker, out)

## Mehrere Tage vergleichen

Um den Verlauf zweier Wochentage zu vergleichen, müssen wir den Befehl `iplot` durch den Parameter `data` ergänzen.

Dazu legen wir eine Liste mit Namen `data` an, die die **Spuren** (engl. Traces) beinhaltet, welche wir zeichnen möchten. Damit die Daten übereinander und nicht hintereinander gezeichnet werden, müssen wir uns auf die Uhrzeit des `TimeStamp` beschränken.

**Beachte:** Pandas behandelt den Datentyp `datetime.time` nicht wie ein vollständiges Datum (`datetime`), sondern wie eine Zeichenkette. Daher wird der n-te Wert einfach an die n-te Stelle geschrieben. Vergleiche dazu die x-Achse der folgenden Abbildung mit der x-Achse der Vorherigen. In unserem Fall ist das jedoch unproblematisch, da die Abstände zwischen zwei Messzeitpunkten immer genau eine Stunde betragen.

In [ ]:
sonntag = df['2017-03-05']

# Definiere fuer Sonntag und Montag jeweils eine Spur
data = [
    {
        'x': sonntag.index.time,
        'y': sonntag['P1AK'],
        'name': 'Sonntag der 05.03.2017'
    },
    {
        'x': montag.index.time,
        'y': montag['P1AK'],
        'name': 'Montag der 06.03.2017'
    }
]

df.iplot(
    data=data,
    title='Montag und Sonntag im Vergleich',
    xTitle='Uhrzeit',
    yTitle='Belegung'
)

Wir können das Problem auch umgehen, indem wir jede Uhrzeit mit einem Datum versehen. Welches Datum wir dafür wählen spielt keine Rolle. Es muss jedoch beides Mal das gleiche Datum sein.

In [ ]:
# Korrigiert die Urzeiten und versieht sie mit dem heutigen Datum
def correct_time(time):
    return pd.to_datetime(time.astype(str))

# Definiert die Spuren für Montag und Sonntag
data = [
    {
        'x': correct_time(sonntag.index.time),
        'y': sonntag['P1AK'],
        'name': 'Sonntag der 05.03.2017'
    },
    {
        'x': correct_time(montag.index.time),
        'y': montag['P1AK'],
        'name': 'Montag der 06.03.2017'
    }
]

df.iplot(
    data=data,
    title='Montag und Sonntag im Vergleich',
    xTitle='Uhrzeit',
    yTitle='Belegung'

)

<div style="background-color: lightblue; padding: 5px 20px 20px">

### Aufgabe (Parkplatzbelegung an einem Samstag)

Vergleiche den Verlauf der Belegung des P1 an einem **Freitag** mit dem an einem **Samstag**.

Wie **unterscheiden** sich die Graphen?

Was könnten **Gründe** für diese Unterschiede sein?
</div>

In [ ]:
# Hier ist Platz für deinen Code

*Hier ist Platz für deine Antwort*

## Gruppieren und aggregieren

### Daten gruppieren

Zur weiteren Analyse sollen jetzt die Parkplatzbelegungen über alle Montage gemittelt werden. Dazu filtern wir die Daten zunächst, indem wir nur die Einträge auswählen, deren `TimeStamp` an einem Montag ist.

In [ ]:
montage = df[df.index.dayofweek == 0]

Jetzt stehen die Daten aller Montage jeweils von 00:00 Uhr bis 23:00 Uhr zur Verfügung.

Durch den Befehl `groupby` fassen wir alle Einträge mit der **gleichen Uhrzeit** zu **Gruppen** zusammen.

In [ ]:
grouped = montage.groupby(montage.index.time)

Für jede volle Stunde erhalten wir eine **Gruppe**, die die Belegung aller Parkplätze zu dieser Urzeit an allen Montagen des Jahres enthält.

In [ ]:
# Gib die Namen aller Gruppen und die ersten 5 Zeilen jeder Gruppe aus
for name, group in grouped:
    display(str(name))
    display(group.head())

### Daten aggregieren und visualisieren

Anschließend aggregieren wir die Gruppen. Das bedeutet wir berechnen für eine oder mehrere Spalten einer Gruppe eine statistische Kennziffer (z.B. Durchschnitt, Maximum, Median) und können so die Gruppen anhand dieser miteinander vergleichen.

In [ ]:
# Berechne für jede Gruppe (Uhrzeit) die durchschnittliche Belegung
agg = grouped.agg({'P1AK': 'mean'})

# Schönheitskorrektur, damit die Uhrzeiten korrekt dargestellt werden
agg.index = correct_time(agg.index)

agg.iplot(
    title='Durchschnittliche Belegung des P1 an einem Montag',
    xTitle='Uhrzeit',
    yTitle='Belegung'
)

Um ein Gefühl für die Streuung der Daten zu bekommen, macht es Sinn neben dem Durchschnitt auch die **minimale** beziehungsweise die **maximale** Belegung zum jeweiligen Zeitpunkt anzuzeigen.

In [ ]:
agg = grouped.agg({'P1AK': ['min', 'mean', 'max']})

# Schönheitskorrektur, damit die Uhrzeiten korrekt dargestellt werden
agg.index = correct_time(agg.index)

agg.iplot(
    y='P1AK',
    title='Durchschnittliche, minimale und maximale Belegung an einem Montag',
    xTitle='Uhrzeit',
    yTitle='Belegung'
)

Unter zu Hilfenahme von `interact` erhalten wir auf diese Weise für jeden Wochentag einen enstprechenden Graphen.

In [ ]:
woche = ['Montag', 'Dienstag', 'Mittwoch', 'Donnerstag', 'Freitag', 'Samstag', 'Sonntag']

@interact(wochentag=woche)
def plot_wochentage(wochentag):
    tag = df[df.index.dayofweek == woche.index(wochentag)]
    agg = tag.groupby(tag.index.time).agg({'P1AK': ['min', 'mean', 'max']})
    agg.index = correct_time(agg.index)
    agg.iplot(y='P1AK')

<div style="background-color: lightblue; padding: 5px 20px 20px">

### Aufgabe (Interaktive Darstellung)

Ändere den obigen Code so, dass auch der **Parkplatz** über ein **Dropdown-Menü** (`interact`) ausgewählt werden kann.

**Tipp:** Mit dem Befehl
```python
df.columns.tolist()
```
erhälst du eine Liste aller Spaltennamen.
</div>

In [ ]:
# Hier ist Platz für deine Lösung

## Wochentage im Vergleich

Um die verschiedene Wochentage besser miteinander zu **vergleichen**, benötigen wir eine Darstellung, in der jeder Wochentag simultan durch eine Spur angezeigt wird.

Dazu verallgemeinern wir unsere Vorgehensweise und legen für jeden Wochentag eine Spur an. 

In [ ]:
# Beginne mit einer leeren Liste
data = []

# Für jeden Wochentag in der Woche
for wochentag in woche:
    # Selektiere alle Einträge an dem entsprechenden Wochentag
    tag = df[df.index.dayofweek == woche.index(wochentag)]
    tag = tag.groupby(tag.index.time).agg('P1AK').mean()
    # Füge die Spur für den jeweiligen Tag hinzu
    data.append(
        {
            'x': correct_time(tag.index),
            'y': tag,
            'name': wochentag
        }
    )
    
df.iplot(
    data=data,
    title='Wochentage im Vergleich (P1)',
    xTitle='Uhrzeit',
    yTitle='Belegung'
)

## Berücksichtigung von Feiertagen

Wir haben die Vermutung, dass sich die Parkplatzbelegungen an **Feiertagen** und **Werktagen** unterscheiden. Um die Vermutung zu überprüfen schauen wir uns zunächst exemplarisch den Ostermontag (17.04.2017) an.

In [ ]:
ostermontag = df['2017-04-17']

data = [
    {
        'x': correct_time(ostermontag.index.time),
        'y': ostermontag['P1AK'],
        'name': 'Ostermontag'
    },
    {
        'x': correct_time(montag.index.time),
        'y': montag['P1AK'],
        'name': 'Werkmontag'
    }
]

df.iplot(
    data=data,
    title='Ostermontag und Werkmontag im Vergleich',
    xTitle='Uhrzeit',
    yTitle='Belegung'
)

Tatsächlich unterscheiden sich die Graphen deutlich voneinander.

Im Folgendem wollen wir daher die Feiertage und Nicht-Feiertage separat betrachten.

Gesetzliche Feiertage 2017 im deutschen Bundesland Nordrhein-Westfalen:

| Name | Wochentag | Datum |
| :--- | :--- | :--- |
| Neujahr             |	Sonntag    | 01.01.2017 |
| Karfreitag          | Freitag    | 14.04.2017 |
| Ostermontag         |	Montag     | 17.04.2017 |
| Tag der Arbeit      | Montag     | 01.05.2017 |
| Christi Himmelfahrt |	Donnerstag | 25.05.2017 |
| Pfingstmontag       | Montag     | 05.06.2017 |
| Fronleichnam        |	Donnerstag | 15.06.2017 |
| Tag der Deutschen Einheit | Dienstag | 03.10.2017 |
| Reformationstag           | Dienstag | 31.10.2017 | 
| Allerheiligen             | Mittwoch | 01.11.2017 |
| 1. Weihnachtsfeiertag     | Montag   | 25.12.2017 |
| 2. Weihnachtsfeiertag     | Dienstag | 26.12.2017 |

Mit Hilfe des `isin`-Befehls erhalten wir ein `DataFrame`, das alle Feiertage enthält und ein `DataFrame`, das alle Nicht-Feiertage (im Folgendem zu Gunsten der Lesbarkeit: "Werktage") enthält.

In [ ]:
liste_feier = [
    '2017-01-01',
    '2017-04-14',
    '2017-04-17',
    '2017-05-01',
    '2017-05-25',
    '2017-06-05',
    '2017-06-15',
    '2017-10-03',
    '2017-10-31',
    '2017-11-01',
    '2017-12-25',
    '2017-12-26'
]

liste_feier = pd.to_datetime(liste_feier)

# Wähle alle Einträge, deren Index in der Liste der Feiertage liegt
feiertage = df[np.isin(df.index.date, liste_feier.date)]
# Analog, aber Abfrage negiert (~)
werktage = df[~np.isin(df.index.date, liste_feier.date)]

Auf die uns bekannte Weise filtern wir die **Montage** aus den Daten.

In [ ]:
werkmontage = werktage[werktage.index.dayofweek==0]
feiermontage = feiertage[feiertage.index.dayofweek==0]

Schließlich gruppieren wir erneut über die Uhrzeiten und aggregieren mittels Durchschnitt.

In [ ]:
# Fasse die Einträge mit der selber Uhrzeit zu Gruppen zusammen
werkmontage_grouped = werkmontage.groupby(werkmontage.index.time)
feiermontage_grouped = feiermontage.groupby(feiermontage.index.time)

# Bilde den Durchschnitt, das Minimum und Maximum der Gruppen
werkmontage_agg = werkmontage_grouped.agg({'P1AK': 'mean'})
feiermontage_agg = feiermontage_grouped.agg({'P1AK': 'mean'})

data = [
    {
        'x': correct_time(werkmontage_agg.index),
        'y': werkmontage_agg['P1AK'],
        'name': 'Werkmontage'
    },
    {
        'x': correct_time(feiermontage_agg.index),
        'y': feiermontage_agg['P1AK'],
        'name': 'Feiermontage'
    }
]

df.iplot(
    data=data,
    title='Feiermontage und Nicht-Feiermontage im Vergleich (P1)',
    xTitle='Uhrzeit',
    yTitle='Belegung'
)

## Jahresüberblick

Schließlich wollen wir uns noch mit der Frage beschäftigen, in welchen Kalenderwochen des Jahres der Parkplatz P1 besonders voll war.

Mit der Funktion
```python
resample(rule=...)
```
können wir Einträge, die innerhalb der selben Stunde, Woche, Monat etc. liegen, zusammenfassen.

Anschließend können wir über diese Intervalle aggregieren, indem wir den Durchschnitt bilden. Dadurch erhalten wir eine Übersicht über die Belegung des Parkplatzes im Verlauf des Jahres.

In [ ]:
# Fasse alle Einträge, die innerhalb der selben Woche (1W) liegen, zusammen und bilde deren Durchschnitt
df_overview = df.resample(rule='1W').mean()

df_overview.iplot(
    y='P1AK',
    kind='bar',
    title='Belegung des P1 2017',
    xTitle='Kalenderwoche',
    yTitle='Durchschnittliche Belegung'
)

<div style="background-color: lightblue; padding: 5px 20px 20px">

### Aufgabe (Interpretation)

Betrachte den Graphen und beantworte **kurz** folgende Fragestellungen:
+ In welchen Wochen war der Parkplatz besonders voll bzw. leer? Was könnten Gründe dafür sein?
+ Was sollte man in der gewählten Darstellung im Hinblick auf den Jahreswechsel beachten?

</div>

*Hier ist Platz für deine Antwort*